In [ ]:
# Transformer 모델 구축 - Transformer 대화형 챗봇(Dialogue Chatbot) 모델
# 학습 목표 - 실무에서 사용되는 파이프라인 이해 및 적용
# - 1. 데이터셋 로드 및 데이터 분리
# - AI Hub 대화데이터: 한국어 SNS 멀티턴 대화 데이터
# - 데이터 분리: train, validation
# - 2. 데이터 분리 및 대화 쌍 만들기
# - train/validation/test 분리
# - utterance 쌍(이전대화, 현재대화) 추출
# - 3. Dataset, DataLoader 생성
# - Hugging Face Dataset -> PyTorch Dataset 클래스로 감싸기
# - DataLoader 생성
# - 4. 모델 정의
# - Feature Extraction + LoRA Fine-tuning 조합
# - 최적화 설정: optimizer, GradScaler, autocast
# - Early Stopping 클래스 정의
# - 최적 모델 가중치 저장
# - 5. 학습/검증 루프
# - 딕셔너리 형태 학습데이터를 그대로 모델에 전달하는 코드로 정리, 코드가 깔끔하고 범용적으로 사용한다
# - Early Stopping 객체 사용하여 적용
# - AMP torch.float32 사용(메모리 사용 증가, torch.float16 사용시 loss가 너무 작아저 nan 발생)
# - 6. 전체 평가 파이프라인
# - F1/EM 평가
# - 7. 모델 정의 및 최적화 모델 로드
# - 8. 멀티 답변 생성
# - 9. 문장 추론: Fast API 호출

In [ ]:
# 데이터셋 로드 
# - AI Hub 대화데이터: 한국어 SNS 멀티턴 대화 데이터
# - 데이터 분리: train, validation
from datasets import load_dataset
import glob, random

train_files = glob.glob('llm_data/ai_hub_dialogue_session2/train/*.json')
valid_files = glob.glob('llm_data/ai_hub_dialogue_session2/validation/*.json')

# 6%만 샘플링
sample_train = random.sample(train_files, int(len(train_files) * 0.06))
sample_valid = random.sample(valid_files, int(len(valid_files) * 0.03))

dataset = load_dataset(
    'json',
    data_files={
        'train': sample_train,
        'validation': sample_valid
    },
    field='sessionInfo'
)
print(dataset['train'].shape)

Resolving data files:   0%|          | 0/1985 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/124 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

(3970, 10)


In [7]:
# 데이터 분리 및 대화 쌍 만들기
# - train/validation/test 분리
# - utterance 쌍(이전대화, 현재대화) 추출

# 대화 파싱 함수: map(batched=True)로 여러 쌍을 한번에 반환 할 수 있다
def extract_pairs(batch):
    contexts = []
    responses = []
    for example in batch['dialog']:
        for i in range(1, len(example)):
            contexts.append(example[i-1]['utterance']) # 이전 대화
            responses.append(example[i]['utterance']) # 현재 대화
    return {
        'context': contexts,
        'response': responses
    }

# utterance 쌍 추출, map 사용: 리스트 컬럼 생성
train_pairs = dataset['train'].map(
    extract_pairs, 
    remove_columns=dataset['train'].column_names,
    batched=True
)
valid_pairs = dataset['validation'].map(
    extract_pairs, 
    remove_columns=dataset['validation'].column_names,
    batched=True
)

# flatten으로 리스트 풀기
train_pairs = train_pairs.flatten()
valid_pairs = valid_pairs.flatten()

# validation -> validation/test 분리
split_dataset = valid_pairs.train_test_split(test_size=0.5, seed=42)
valid_pairs = split_dataset['train']
test_pairs = split_dataset['test']

print(train_pairs[0])
print(train_pairs.shape)

Map:   0%|          | 0/3970 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

{'context': '안녕하세요! 요즘 부모님과 마찰이 생기는 일이 많아서 힘드네요.', 'response': '그럴 때는 상황을 잘 다루는 것이 중요한 것 같아요. 어떤 상황에서 마찰이 생기는지 알려주세요.'}
(58173, 2)


In [8]:
# Dataset, DataLoader 생성
# - Hugging Face Dataset -> PyTorch Dataset 클래스로 감싸기
# - DataLoader 생성
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

# 토크나이저 준비: MT5 모델
tokenizer = AutoTokenizer.from_pretrained('google/mt5-small')

class DialogueDataset(Dataset):
    def __init__(self, hf_dataset, tokenizer, max_length=64): # 파라미터 초기 셋팅
        self.dataset=hf_dataset
        self.tokenizer=tokenizer
        self.max_length=max_length
    
    def __len__(self): # dataset 길이 리턴
        return len(self.dataset)
    
    def __getitem__(self, idx):
        item = self.dataset[idx]
        context = item['context']
        response = item['response']

        # 토크나이즈
        inputs = self.tokenizer(
            context,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        labels = self.tokenizer(
            response,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )["input_ids"].squeeze(0)

        # pad_token_id → -100 (loss 계산에서 무시)
        labels[labels == self.tokenizer.pad_token_id] = -100

        # <extra_id_0> 제거 (선택)
        extra_id_0 = self.tokenizer.convert_tokens_to_ids("<extra_id_0>")
        if labels[0] == extra_id_0:
            labels = labels[1:]

        return {
            'input_ids': inputs['input_ids'].squeeze(0), # 불필요한 squeeze(0) 앞 배치 차원만 제거
            'attention_mask': inputs['attention_mask'].squeeze(0),
            'labels': labels
        }

# Dataset 생성
train_dataset = DialogueDataset(train_pairs, tokenizer)
valid_dataset = DialogueDataset(valid_pairs, tokenizer)
test_dataset = DialogueDataset(test_pairs, tokenizer)

# DataLoader 생성
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=16,
    shuffle=True
)
valid_loader = DataLoader(
    dataset=valid_dataset,
    batch_size=16
)
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=16
)

# 데이터 확인
batch = next(iter(train_loader))
print(batch['input_ids'].shape) # torch.Size([32, 128])
print(batch['attention_mask'].shape) # torch.Size([32, 128])
print(batch['labels'].shape) # torch.Size([32, 128])

print("학습 데이터 개수:", len(train_dataset))
print("검증 데이터 개수:", len(valid_dataset))
print("배치 크기:", train_loader.batch_size)
print("학습 로더 길이:", len(train_loader))
print("검증 로더 길이:", len(valid_loader))


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
d:\AI\Pytorch\deep-learning\.venv\Lib\site-packages\transformers\convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


torch.Size([16, 64])
torch.Size([16, 64])
torch.Size([16, 64])
학습 데이터 개수: 58173
검증 데이터 개수: 1826
배치 크기: 16
학습 로더 길이: 3636
검증 로더 길이: 115


In [9]:
# 모델 정의
# - 모델 본체 동결 처리: Feature Extraction
# - LoRA 파인튜닝 적용
# - Early Stopping 적용
import torch
from transformers import AutoModelForSeq2SeqLM
from peft import LoraConfig, get_peft_model
from torch.amp import GradScaler, autocast
import os

model = AutoModelForSeq2SeqLM.from_pretrained('google/mt5-small') # MT5 모델 불러온다

# GPU 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'PyTorch Version: {torch.__version__}, Device: {device}')

# 모델 전체를 GPU/CPU 디바이스 메모리로 이동
model = model.to(device)

# 모델 본체 동결 처리: Feature Extraction
for param in model.parameters():
    param.requires_grad=False

# # LoRA 대상 모듈 자동 추출
# target_modules = []
# for name, module in model.named_modules():
#     if any(x in name for x in ["SelfAttention.q", "SelfAttention.k", "SelfAttention.v", "SelfAttention.o"]):
#         target_modules.append(name)

# print("LoRA target modules:", target_modules)

# LoRA 적용: LoRA 모듈만 학습되도록 설정(경량 파인튜닝)
lora_config = LoraConfig(
    r=8, # 작은 rank(r=8)로 효율적인 파인튜닝 가능
    lora_alpha=32, # LoRA scaling factor
    # target_modules=['q', 'v'], # Attention 모듈의 Query/Value 부분에 LoRA 레이어 추가
    target_modules=['q', 'v'], # Attention 모듈의 Query/Value 부분에 LoRA 레이어 추가
    lora_dropout=0.1, # 드롭아웃
    bias='none',
    task_type='SEQ_2_SEQ_LM' # 대화 응답 생성은 Seq2Seq LM
)
# LoRA 모델 생성
model = get_peft_model(model, lora_config)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
# Automatic Mixed Precision(AMP) 학습을 위한 GradScaler 준비
# Automatic Mixed Precision(AMP)은 모델 파라미터는 FP32로 유지하면서 연산(곱셈·덧셈 등)만 FP16으로 자동 전환하여, 
# 정밀도 손실 없이 메모리와 연산 효율을 극대화하는 기술
scaler = GradScaler()
num_epochs = 3

# Early Stopping
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0.0, path='./llm_models/21_transformer_dialogue_chatbot/best_model.pt'):
        self.patience=patience
        self.min_delta=min_delta
        self.best_loss=None
        self.counter=0
        self.early_stop=False
        self.path=path
    
    def __call__(self, valid_loss, model):
        # 최조 손실값에 해당하는 모델 가중치 저장
        if self.best_loss is None:
            self.best_loss=valid_loss
            self.save_checkpoint(model)
        
        # 성능 개선 -> 최적 모델 갱신
        elif valid_loss < self.best_loss - self.min_delta:
            self.best_loss=valid_loss
            self.counter=0
            self.save_checkpoint(model)
        
        # 개선 없음
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop=True
    
    def save_checkpoint(self, model):
        # 디렉토리만 생성
        folder = os.path.dirname(self.path)
        if folder !='' and not os.path.exists(folder):
            os.makedirs(folder)
        # 모델 가중치 저장
        torch.save(model.state_dict(), self.path)
        print(f' Best model saved at {self.path}')

# Early Stopping 객체 생성
early_stopping = EarlyStopping(patience=3, min_delta=0.001)

# 모델 확인
print(model)

PyTorch Version: 2.8.0+cu129, Device: cuda
PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): MT5ForConditionalGeneration(
      (shared): Embedding(250112, 512)
      (encoder): MT5Stack(
        (embed_tokens): Embedding(250112, 512)
        (block): ModuleList(
          (0): MT5Block(
            (layer): ModuleList(
              (0): MT5LayerSelfAttention(
                (SelfAttention): MT5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=512, out_features=384, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=512, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=384, bias=False)
                    )
                    (

In [ ]:
# - 학습/검증 루프
# - 딕셔너리 형태 학습데이터를 그대로 모델에 전달하는 코드로 정리, 코드가 깔끔하고 범용적으로 사용한다
# - Early Stopping 객체 사용하여 적용
# - AMP torch.float32 사용(메모리 사용 증가, torch.float16 사용시 loss가 너무 작아저 nan 발생)

from tqdm import tqdm

for epoch in range(num_epochs):
    # Train
    model.train() # 학습 모드 지정
    total_train_loss = 0

    # Train Loop
    for batch in tqdm(train_loader, desc=f'Epoch {epoch+1} [Train]'):
        batch = { k: v.to(device) for k, v in batch.items() } # 딕셔너리 형태로 생성, 학습데이터 GPU 지정
        optimizer.zero_grad() # 오차역전파 코드, 미분 전 가중치/바이어스 파라미터 초기화

        # AMP(Automatic Mixed Precision) GPU에서 연산 속도와 메모리 효율 향상
        # with autocast(device_type='cuda', dtype=torch.float16):
        #     outputs = model(**batch) # 딕셔너리 형태 학습데이터를 그대로 모델에 전달, 코드가 깔끔하고 범용적으로 사용한다
        #     loss = outputs.loss # 손실값
        # scaler.scale(loss).backward() # 미분 연산
        # scaler.step(optimizer) # 미분 연산 후 가중치/바이어스 파라미터 업데이트
        # scaler.update()

        with autocast(device_type='cuda', enabled=False):
            outputs = model(**batch)
            loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item() # 손실 누적
    avg_train_loss = total_train_loss / len(train_loader)
    print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}')

    # Validation Loop
    model.eval() # 검증/추론 모드 지정
    total_val_loss = 0

    with torch.no_grad():
        for batch in tqdm(valid_loader, desc=f'Epoch {epoch+1} [Valid]'):
            batch = { k: v.to(device) for k, v in batch.items() }

            # AMP(Automatic Mixed Precision) GPU에서 연산 속도와 메모리 효율 향상
            # with autocast(device_type='cuda', dtype=torch.float16):
            #     outputs = model(**batch) # 검증 모델 예측
            #     loss = outputs.loss # 검증 손실값

            with autocast(device_type='cuda', enabled=False):
                outputs = model(**batch)
                loss = outputs.loss

            total_val_loss += loss.item() # 검증 손실값 누적    
    avg_val_loss = total_val_loss / len(valid_loader)
    print(f'Epoch {epoch+1}, Valid Loss: {avg_val_loss:.4f}')

    # Early Stopping 체크
    early_stopping(valid_loss=avg_val_loss, model=model)
    if early_stopping.early_stop:
        print('Early stopping triggered.')
        break

Epoch 1 [Train]: 100%|██████████| 3636/3636 [2:20:32<00:00,  2.32s/it]  


Epoch 1, Train Loss: 7.9773


Epoch 1 [Valid]: 100%|██████████| 115/115 [00:57<00:00,  1.99it/s]


Epoch 1, Valid Loss: 4.1618
 Best model saved at ./llm_models/21_transformer_dialogue_chatbot/best_model.pt


Epoch 2 [Train]: 100%|██████████| 3636/3636 [2:20:31<00:00,  2.32s/it]  


...

Epoch 2, Valid Loss: 3.8294
 Best model saved at ./llm_models/21_transformer_dialogue_chatbot/best_model.pt


Epoch 3 [Train]: 100%|██████████| 3636/3636 [2:20:33<00:00,  2.32s/it]  


Epoch 3, Train Loss: 4.7710


Epoch 3 [Valid]: 100%|██████████| 115/115 [00:57<00:00,  1.99it/s]


Epoch 3, Valid Loss: 3.4802
 Best model saved at ./llm_models/21_transformer_dialogue_chatbot/best_model.pt


In [ ]:
# 전체 평가 파이프라인: F1/EM 평가


In [11]:
# 모델 정의 및 최적화 모델 로드

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import LoraConfig, get_peft_model

# GPU 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'PyTorch Version: {torch.__version__}, Device: {device}')

# 같은 구조의 모델 초기화
tokenizer = AutoTokenizer.from_pretrained('google/mt5-small')
model = AutoModelForSeq2SeqLM.from_pretrained('google/mt5-small') # MT5 모델 불러온다

# 모델 본체 동결 처리: Feature Extraction
for param in model.parameters():
    param.requires_grad=False
        
# LoRA 적용: LoRA 모듈만 학습되도록 설정(경량 파인튜닝)
lora_config = LoraConfig(
    r=8, # 작은 rank(r=8)로 효율적인 파인튜닝 가능
    lora_alpha=32, # LoRA scaling factor
    # target_modules=['q', 'v'], # Attention 모듈의 Query/Value 부분에 LoRA 레이어 추가
    target_modules=['q', 'v'], # Attention 모듈의 Query/Value 부분에 LoRA 레이어 추가
    lora_dropout=0.1, # 드롭아웃
    bias='none',
    task_type='SEQ_2_SEQ_LM' # 대화 응답 생성은 Seq2Seq LM
)
# LoRA 모델 생성
model = get_peft_model(model, lora_config)

# 저장된 state_dict 불러오기
model.load_state_dict(torch.load('./llm_models/21_transformer_dialogue_chatbot/best_model.pt'))

# 추론/검증 모드 적용
model.eval()

# 모델 전체를 GPU/CPU 디바이스 메모리로 이동
model = model.to(device)

# 모델 확인
print(model)

PyTorch Version: 2.8.0+cu129, Device: cuda
PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): MT5ForConditionalGeneration(
      (shared): Embedding(250112, 512)
      (encoder): MT5Stack(
        (embed_tokens): Embedding(250112, 512)
        (block): ModuleList(
          (0): MT5Block(
            (layer): ModuleList(
              (0): MT5LayerSelfAttention(
                (SelfAttention): MT5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=512, out_features=384, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=512, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=384, bias=False)
                    )
                    (

In [30]:
# 멀티 답변 생성

import re

# context = "서울에서 유명한 관광지 알려줄래?"
context = "서울의 유명 관광지는 경복궁, 남산타워, 명동입니다."

inputs = tokenizer(
    context,
    return_tensors='pt'
).to(device)

# Beam search + Sampling 혼합
outputs = model.generate(
    **inputs,
    max_length=128,
    min_length=8, # 최소 길이 강제

    num_beams=5, # 5~7 정도가 대화형 모델에서는 가장 균형이 좋다
    early_stopping=True,
    do_sample=True,
    top_p=0.85, # 0.9 → 0.6로 줄여서 불필요한 변형을 줄인다, 변형 단어 출력 감소
    temperature=0.8, # 0.7 → 0.5 정도로 낮추면 더 안정적인 답변을 얻을 수 있다
    num_return_sequences=3,

    repetition_penalty=2.0, # 반복 억제
    no_repeat_ngram_size=4, # 4-gram 반복 금지
    length_penalty=1.2 # 1.2~2.0 정도로 설정하면, 모델이 불필요하게 길게 반복하는 걸 줄인다
)

for i, output in enumerate(outputs):
    decoded = tokenizer.decode(output, skip_special_tokens=True)

    # <extra_id_n> 토큰 제거
    decoded = re.sub(r'<extra_id_\d+>', '', decoded).strip()
    decoded = decoded.replace('..', '.').replace('??', '?')
    decoded = re.sub(r'북산타워|서산타워|여산타워|한산타워', '남산타워', decoded)
    decoded = re.sub(r'^,?\s*의 유명 관광지', '서울의 유명 관광지는', decoded)
    decoded = re.sub(r'(국립공원|남산타워)(,\s*\1)+', r'\1', decoded)  # 중복 제거
    decoded = re.sub(r'궁금한 관광지입니다', '관광 명소로 잘 알려져 있습니다', decoded)

    print(f'답변 {i+1}: {decoded}')

답변 1: , 남산타워 등의 유명 관광지가 경복궁입니다.
답변 2: , 남산타워 등의 유명 관광지가 경복궁입니다. 흥미로운 관광지가 있어요.
답변 3: , 남산타워 등의 유명 관광지가 경복궁입니다. 흥미로운 관광지가 있나요?
